In [1]:
import sys
sys.path.insert(0, "/home/GAFOL/work/kspec_gfa_controller/src")  # <-- add *src*, not the package folder

from kspec_gfa_controller.gfa_actions import GFAActions



In [2]:
from astropy.io import fits

def print_ra_dec(fits_path):
    with fits.open(fits_path) as hdul:
        hdr = hdul[0].header
        ra = hdr.get('RA')
        dec = hdr.get('DEC')
        print(f"RA: {ra}, DEC: {dec}")

# 사용 예
#print_ra_dec("/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/raw/D20260123_T152557_40103667_exp5s.fits")


In [3]:
actions = GFAActions()

2026-01-24 19:00:04,867 [INFO] Initialized GFAEnvironment with role 'plate' and cameras [1, 2, 3, 4, 5, 6]
2026-01-24 19:00:04,867 [INFO] Initializing GFAController with provided config.
2026-01-24 19:00:04,868 [INFO] GFAController initialization complete.
2026-01-24 19:00:04,868 [INFO] Initializing gfa_astrometry class.
2026-01-24 19:00:04,868 [INFO] Paths:
2026-01-24 19:00:04,868 [INFO]   raw_images        = /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/raw
2026-01-24 19:00:04,869 [INFO]   temp_files        = /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/tempfiles
2026-01-24 19:00:04,869 [INFO]   final_astrometry  = /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/astroimg/
2026-01-24 19:00:04,869 [INFO]   star_catalog_dir  = /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/star_catalog
2026-01-24 19:00:04,869 [INFO]   combined_star.fits= /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/st

In [4]:
#await actions.grab(ExpTime=0.05) # RA, DEC 

In [5]:
# ra="05:49:42.20", dec="-45:47:02.28"
await actions.pointing(ra="05:49:42.20", dec="-45:47:02.28", save=True) # RA, DEC

2026-01-24 19:00:04,876 [INFO] Starting pointing sequence...
2026-01-24 19:00:04,877 [INFO] Target RA/DEC: 05:49:42.20, -45:47:02.28
2026-01-24 19:00:04,877 [INFO] Deleting raw files.
2026-01-24 19:00:04,877 [INFO] Deleted 0 raw files.
2026-01-24 19:00:04,877 [INFO] Opening all cameras...
2026-01-24 19:00:05,416 [INFO] Cam6 opened (IP 192.168.88.6).
2026-01-24 19:00:05,427 [INFO] Cam2 opened (IP 192.168.88.2).
2026-01-24 19:00:05,438 [INFO] Cam5 opened (IP 192.168.88.5).
2026-01-24 19:00:05,449 [INFO] Cam1 opened (IP 192.168.88.1).
2026-01-24 19:00:05,460 [INFO] Cam3 opened (IP 192.168.88.3).
2026-01-24 19:00:05,471 [INFO] Cam4 opened (IP 192.168.88.4).
2026-01-24 19:00:05,471 [INFO] All cameras opened successfully.
2026-01-24 19:00:06,831 [WARNING] No date_obs provided. Using current date.
2026-01-24 19:00:06,831 [WARNING] No time_obs provided. Using current time.
2026-01-24 19:00:07,085 [INFO] FITS file successfully saved to /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_control

{'status': 'success',
 'message': 'Pointing completed. Computed CRVALs for 6 images.',
 'images': ['astro_D20260124_T100006_40103834_exp1s.fits',
  'astro_D20260124_T100007_40103651_exp1s.fits',
  'astro_D20260124_T100007_40103667_exp1s.fits',
  'astro_D20260124_T100007_40103831_exp1s.fits',
  'astro_D20260124_T100007_40103833_exp1s.fits',
  'astro_D20260124_T100008_40103663_exp1s.fits'],
 'crval1': [85.5677631684,
  87.4376158281,
  89.2775002964,
  89.2148281688,
  85.5998461175,
  87.3993115747],
 'crval2': [-46.5150226152,
  -47.2563765753,
  -46.493666282,
  -45.0242255581,
  -45.0410085881,
  -44.310185723]}

In [8]:
# ra="05:49:42.20", dec="-45:47:02.28"
await actions.guiding(ra="05:49:42.20", dec="-45:47:02.28") # RA, DEC

2026-01-24 19:00:23,443 [INFO] Deleting raw files.
2026-01-24 19:00:23,443 [INFO] Deleted 0 raw files.
2026-01-24 19:00:23,443 [INFO] Starting guiding sequence...
2026-01-24 19:00:23,444 [INFO] Opening all cameras...
2026-01-24 19:00:23,963 [INFO] Cam1 opened (IP 192.168.88.1).
2026-01-24 19:00:23,972 [INFO] Cam5 opened (IP 192.168.88.5).
2026-01-24 19:00:23,980 [INFO] Cam3 opened (IP 192.168.88.3).
2026-01-24 19:00:23,990 [INFO] Cam4 opened (IP 192.168.88.4).
2026-01-24 19:00:23,999 [INFO] Cam2 opened (IP 192.168.88.2).
2026-01-24 19:00:24,008 [INFO] Cam6 opened (IP 192.168.88.6).
2026-01-24 19:00:24,009 [INFO] All cameras opened successfully.
2026-01-24 19:00:25,369 [WARNING] No date_obs provided. Using current date.
2026-01-24 19:00:25,369 [WARNING] No time_obs provided. Using current time.
2026-01-24 19:00:25,618 [INFO] FITS file successfully saved to /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/raw/D20260124_T100025_40103831_exp1s.fits
2026-01-24 19:00:25,620

{'status': 'success',
 'message': 'Offsets: fdx=1.076081898932011, fdy=-0.014447027678652628, FWHM=3.8860270198234317 arcsec',
 'fdx': 1.076081898932011,
 'fdy': -0.014447027678652628,
 'fwhm': 3.8860270198234317,
 'astrometry_files': ['astro_D20260124_T100006_40103834_exp1s.fits',
  'astro_D20260124_T100007_40103651_exp1s.fits',
  'astro_D20260124_T100007_40103667_exp1s.fits',
  'astro_D20260124_T100007_40103831_exp1s.fits',
  'astro_D20260124_T100007_40103833_exp1s.fits',
  'astro_D20260124_T100008_40103663_exp1s.fits']}

In [7]:
#solve-field 
# ./clean/D20260119_T133618_40103834_exp1s_hotpix_1p5med.fits   
# -D /tmp/gfa_xyls_solve   -O --no-plots --no-verify --crpix-center   
# -X X -Y Y -s FLUX   --scale-units degwidth -L 0.12 -H 0.22   
# --ra "06:57:31.68" --dec "-29:06:02.30" --radius 2   -l 120  -c 0.1 -E 2